# Musical features extraction
Owing to the type of data that a machine learning need to be fed with, this process will be apply to every selected song.

### Libraries

In [ ]:
import librosa
import numpy as np
import essentia.standard as ess
import pandas as pd
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler

### Data paths

In [ ]:
parent_path = 'PRDL+MLLB'
data_path = f'{parent_path}/used_dataset'
genres = [x for x in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, x))]
# This names will be the columns names if the dataset that the machine learning algorithm will have as input
freq_features = [f"MFCC {num}" for num in range(1, 21)]
others_features = ['Beats_song', 'Danceability', 'Loudness', 'Spectral_Rolloff', 'Spectral Centroid', 'Energy']
features = others_features + freq_features

### Dataframe creation and 

In [ ]:
df = pd.DataFrame(columns=["Genre"] + features)

### Features extraction and dataframe addition

In [ ]:
for genre in genres:
    # Music path
    music = f'{data_path}/{genre}'
    songs = os.listdir(music)
    print(f'Number of files in {genre} folder: {len(songs)}')
    for song in songs:
        file, extension = os.path.splitext(song)
        if not extension == '.mp3':
            print(f'The {genre} folder not only contains songs')
            print(f'\tNot valid file: {file}')
            songs.remove(file)
            continue

        # Feature extraction
        audio, sr = librosa.load(f'{music}/{song}')

        # MFCC
        mfccs_values = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)
        MFCCs = np.mean(mfccs_values, axis=1)

        # Tempo and beats per song
        # Tempo won't be use because its poor correlation to the genre
        tempo, beat_frames = librosa.beat.beat_track(y=audio, sr=sr)
        beats = librosa.frames_to_time(beat_frames, sr=sr)
        total_beats = len(beats)

        # Loudness
        loudness = ess.Loudness()(audio)
        loudness = 10 * np.log10(loudness)

        # Danceability
        danceability = ess.Danceability()(audio)

        # Energy
        # energy = ess.Energy()(audio)
        # energy = 10 * np.log10(energy)

        # Spectrum
        # Check if the length is odd
        if len(audio) % 2 != 0:
            audio = np.append(audio, 0)

        spectrum = ess.Spectrum()(audio)

        # Spectral Roll-off
        roff = ess.RollOff()(spectrum)

        # Spectral Centroid
        centroid = ess.Centroid()(spectrum)

        # Regression dataframe
        new_row_r = MFCCs.tolist()
        # Classification dataframe
        new_row_c = [genre, total_beats, danceability, loudness, roff, centroid]

        # Whole dataframe
        df.loc[len(df.index), :] = new_row_c + new_row_r

### Save the dataframe

In [ ]:
df_path = f'{str(Path(Path.cwd()))}'
df.to_csv(f'{df_path}/df_features.csv', sep=';', decimal=",", index=False)

### # Normalise each column with Z-score (mean = 0, std = 1) and save the new dataframe

In [ ]:
standard_scaler = StandardScaler()
# General dataset
for feat in features:
    df[feat] = standard_scaler.fit_transform(df[[feat]])

# Normalised dataframe
df_path = f'{str(Path(Path.cwd()))}'
df.to_csv(f'{df_path}/df_norm.csv', sep=';', decimal=",", index=False)